In [ ]:
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse
# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()
# import getpass
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
# !mkdir -p drive
# !google-drive-ocamlfuse drive

In [1]:
import os 
import sys
import tensorflow as tf
import pandas as pd
import numpy as np

from tensorflow.python.keras import models
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.callbacks import ModelCheckpoint


from sklearn import model_selection
from __future__ import absolute_import

# os.chdir("/content/drive/Hamming")
# sys.path.append("Hamming")
# import functions as f
# import model_lib 

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
tf.test.gpu_device_name()

In [ ]:
!ls

In [ ]:
# !git pull

In [2]:
TRAIN_PATH = './dataset_files/hamming.txt'  # "/content/drive/Hamming/dataset_files/hamming.txt"

COLUMN_NAMES = ['id', 'plainword', 'codeword', 
                'id_error', 'bin_error', 'defective_codeword']

In [3]:
def hamming_distance(first: str, second: str) -> int:
    return len([1 for (x, y) in zip(first, second) if x != y])

In [4]:
def loss_hamming(y, y_pred):  # , w):
    return sum(1 for (a,b) in zip(y, y_pred) if y != y_pred) / y.shape[0] # надо ли делить????

In [5]:
def load_data():
    data = pd.read_csv(TRAIN_PATH, sep=';', names=COLUMN_NAMES)

    # make features
    data['dec_defective_codeword'] = data['defective_codeword'][:].apply(lambda x: int(x, 2))
    for j in range(len(data['defective_codeword'][0])):
        data['bin_' + str(j)] = data['defective_codeword'][:].apply(lambda x: int(x[j]))
    data['dec_plainword'] = data['plainword'][:].apply(lambda x: int(x, 2))
    return data

In [6]:
def split_data(test_size):
    train_data, test_data, train_labels, test_labels = \
    model_selection.train_test_split(data.loc[:, 'bin_0':'bin_30'], 
                                        data['id_error'], 
                                        test_size = test_size)
    return np.array(train_data), np.array(test_data), np.array(train_labels), np.array(test_labels)
#     return train_data, test_data, train_labels, test_labels

In [7]:
data = load_data() #pd.read_csv(functions.TRAIN_PATH, sep=';', names = functions.COLUMN_NAMES)

In [ ]:
data.head()

In [8]:
def make_features():
    data['dec_defective_codeword'] = data['defective_codeword'][:].apply(lambda x: int(x, 2))

    for j in range(len(data['defective_codeword'][0])):
        data['bin_' + str(j)] = data['defective_codeword'][:].apply(lambda x: int(x[j]))
  
    data['dec_plainword'] = data['plainword'][:].apply(lambda x: int(x, 2))

In [9]:
make_features()

In [ ]:
data.head()

In [10]:
train_data, test_data, train_labels, test_labels = split_data(test_size=0.3)

In [ ]:
# Create feature columns for all features.
feature_columns = []
for key in train_data.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key))

In [ ]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[100, 100, 100],
    activation_fn=tf.nn.relu,
    n_classes=train_data.shape[1]
)

In [ ]:
train_input_fn = tf.estimator.inputs.pandas_input_fn(x=train_data,
                                                    y=train_labels,
                                                    num_epochs=None, #3,
#                                                     batch_size=100,
                                                    shuffle=True
                                                    )

In [ ]:
test_input_fn = tf.estimator.inputs.pandas_input_fn(x=test_data,
                                                    y=test_labels,
                                                    num_epochs=1,
                                                    batch_size=100,
                                                    shuffle=False
                                                    )

In [ ]:
classifier.train(input_fn=train_input_fn, steps=2000)

In [ ]:
result = classifier.evaluate(input_fn=test_input_fn)
print(result)

In [ ]:
def model_fn(features, labels, mode, params):
    h0 = features
#     print(features['bin_0'])
    
    h1 = tf.layers.dense(h0, 100, activation=tf.nn.relu, name="h1")
    h2 = tf.layers.dense(h1, 100, activation=tf.nn.relu, name="h2")
    h3 = tf.layertrain_labelss.dense(h2, 100, activation=tf.nn.relu, name="h3")

    dnn_prediction = tf.layers.dense(h3, 10, name="dnn_prediction")

    loss = tf.losses.mean_squared_error(labels=labels, predictions=dnn_prediction)

    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())

    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

params = {'learning_rate': 1e-3}

# Estimator 2: three hidden layers
estimator_2 = tf.estimator.Estimator(model_fn, params=params, model_dir='./dataset_files/')

estimator_2.train(input_fn=train_input_fn, steps=1000)

In [ ]:
train_input_fn()

In [15]:
model = Sequential()
model.add(Dense(100, input_shape=(31,), activation='relu', name='d1'))
# model.add(Dense(100, activation='relu', name='d2'))
model.add(Dense(31, activation='softmax', name='d3'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy', 'mse'])

Перевести инпут в np.array

model.train(...)

In [16]:
MCP = ModelCheckpoint('best_model.hd5', save_best_only=True)
train_labels_categor = to_categorical(train_labels, num_classes=31)

In [17]:
model.fit(x=train_data, y=train_labels_categor, validation_split=0.2,  shuffle=True, epochs=100, callbacks=[MCP])

Train on 36700 samples, validate on 9175 samples
Epoch 1/100
36700/36700 [==============================]36700/36700 [==============================] - 2s 58us/step - loss: 3.4209 - acc: 0.0333 - mean_squared_error: 0.0312 - val_loss: 3.4087 - val_acc: 0.0340 - val_mean_squared_error: 0.0312

Epoch 2/100
36700/36700 [==============================]36700/36700 [==============================] - 2s 53us/step - loss: 3.4004 - acc: 0.0386 - mean_squared_error: 0.0312 - val_loss: 3.4092 - val_acc: 0.0331 - val_mean_squared_error: 0.0312

Epoch 3/100
36700/36700 [==============================]36700/36700 [==============================] - 2s 54us/step - loss: 3.3916 - acc: 0.0416 - mean_squared_error: 0.0312 - val_loss: 3.4146 - val_acc: 0.0332 - val_mean_squared_error: 0.0312

Epoch 4/100
36700/36700 [==============================]36700/36700 [==============================] - 2s 54us/step - loss: 3.3846 - acc: 0.0450 - mean_squared_error: 0.0311 - val_loss: 3.4193 - val_acc: 0.0347 - val

KeyboardInterrupt: 